# **Setup the development framework such that changes in py files auto load into this file**
- You can ignore this if your interest is only in the business logic and design. 
- The steps in this section are only environment set related

### **Make sure py file change are loaded automatically**

In [ ]:
%load_ext autoreload
%autoreload 2

### **Import basic package required for this assignment**

In [ ]:
import os
import pandas as pd
import db.constants as C
import numpy as np


### **Make sure right sources are getting picked**

In [ ]:
from dao.base_transformer import BaseDBTransformer

### **Make sure the latest class are getting loaded.**

In [ ]:
# 1. Import the modules first (so Python knows them)
import importlib
import service.product_service
import service.cart_service
import service.purchase_service
import service.discount_service
import dao.base_transformer
import dao.cart_transformer
import dao.discount_transformer
import dao.product_transformer
import dao.purchase_transformer
import db.constants
import db.dbsql
import db.dbutil

# 2. Reload them (useful after edits)
importlib.reload(service.product_service)
importlib.reload(service.cart_service)
importlib.reload(service.purchase_service)
importlib.reload(service.discount_service)
importlib.reload(db.constants)
importlib.reload(dao.base_transformer)
importlib.reload(dao.product_transformer)
importlib.reload(dao.cart_transformer)
importlib.reload(dao.discount_transformer)
importlib.reload(dao.purchase_transformer)
importlib.reload(db.dbutil)
importlib.reload(db.dbsql)


# 3. Import the classes fresh from the reloaded modules
from service.product_service import ProductService
from service.cart_service import CartService
from service.purchase_service import PurchaseService
from service.discount_service import DiscountService
from dao.product_transformer import ProductTransformer
from dao.discount_transformer import DiscountTransformer
from dao.cart_transformer import CartTransformer
from dao.purchase_transformer import PurchaseTransformer
from dao.base_transformer import BaseDBTransformer
import db.constants
import random
import db.dbutil
from db.dbsql import SessionLocal, Product
import pandas.testing as pdt
import inspect

# **Transformers**
- This section is the core design of the project
- There are four transformers 
   - Products transformer: This is the database of the list of the products in EStore. The prodcuts.csv file for demo has 9 products. 
   - Cart transformer: This is user selected products added into the cart instance specific to the user. 
   - purchase transfromer: This is checkout functionaility of the cart. 
        - It has two kinds of purchases. With discount by applying discount code.
        - It also supports purchase without applying discount code. If no discount is applied it generates a ne discount code and adds to discounts for the subsequent purchases. 
   - Discount transformer: This handles the discounts codes after the purchase. 

   - For each these transforers there is .py  file and .csv file in the src directory. 
   - Each transfomer derives from the base transfomer class which manages the csv read/write for that transformer. 
   - The base transformer class also handles basis CRUD operation if they are at the level of the dataframe(entire row level operations) without having to deal with individual columns (as columns are specific to the transformer)

- The code is organized as a typical pyproject.toml file. 

## **Product Transformer - demonstration**

#### **Get the product**

In [ ]:
pt = ProductTransformer()
prds_all = pt.list_all_products()
prds_all.head(5)

In [ ]:
prds_all.to_dict(orient="records")

In [ ]:
prds_2050 = pt.list_products_in_class_df(2050)
prds_2050.to_dict(orient='records')

In [ ]:
rows = BaseDBTransformer.read(C.prdc)
rows


#### **Get a single product**

In [ ]:
df = pt.list_all_products(15)
print(type(df))
df

#### **Get the product classes and description**

In [ ]:
prod_classes = pt.get_products_class()

for i,pclass in enumerate(prod_classes[C.ptyp]):
    print("Iteration i ", i, " p class code = ", pclass)
    if ( i>3 ):
        break
    pc_dict = pt.list_products_in_class(pclass)
    print(pc_dict)

prod_classes


## **Discount Transformer - demonstration**

In [ ]:
dt = DiscountTransformer()
dis_all = dt.list_all_codes()
dis_all

## **Purchase Transformer - demonstration**

In [ ]:
prt = PurchaseTransformer()
prt.list_orders_by_customer(3)

## **Cart Transformer - demonstration**

In [ ]:
ct = CartTransformer
ct.list_all()

# **Business logic - Helper functions**

### **Get a random selection of 5 products to purchase**

In [ ]:
products_to_buy = pt.getRandomProducts(5)
customer_id=3


In [ ]:
products_to_buy

### **Add these products to the cart of a selected customer**

##### **Select the items**

In [ ]:
selected_items = products_to_buy.loc[:, [C.pid]]
selected_items[C.qnt] = np.random.randint(1, 3, size=len(selected_items))
items = selected_items.to_dict(orient='records')
items


##### **Add Them to the cart**

In [ ]:
CartTransformer.addToCart(customer_id, items, debug=True)

##### **Verify the addition**

In [ ]:
print("customer_id = ", customer_id)
# CartTransformer.list_cust(customer_id)
# BaseDBTransformer.read(C.cart, **{C.custid:customer_id})
# BaseDBTransformer.readf(C.cart, **{C.custid+"__eq":customer_id})
BaseDBTransformer.read(C.cart, customer_id)

#### **Verify empty cart**

In [ ]:
# PurchaseTransformer.empty_cart(3)

### **Purchase the items in the cart**

In [ ]:
order_id = PurchaseService.purchase_cart_items(customer_id, debug=True)
print("Order Id =", order_id)


##### **Delete a purchase**

In [ ]:
# PurchaseService.deletePurchase(order_id, customer_id)

### **Purchase cart items with discount coupon**

In [ ]:
customer_id = 3
discount_id = DiscountTransformer.generate_discount_id(order_id, customer_id)
print(customer_id, order_id, discount_id)

In [ ]:
cust_cart = BaseDBTransformer.read(C.cart, **{C.custid: customer_id})
for i, dict in enumerate(cust_cart.to_dict(orient='records')):
    prd_df = BaseDBTransformer.read(C.prd, dict[C.pid])
    prd_dict = prd_df.iloc[0].to_dict()
    print(prd_dict)
print(cust_cart)


In [ ]:
print(discount_id)
res_with_d = PurchaseService.purchase_discounted_cart_items(customer_id, discount_id, debug=True)
print( " Order Id with discount =", res_with_d)


In [ ]:
    # lastly cleanup this purchase and verify if delete purcahse is working. 
    order_id_disc = 10072
    cust_id = customer_id
    order_id = 10071
    PurchaseTransformer.deletePurchase(order_id_disc, cust_id, None)
    PurchaseService.deletePurchase(order_id, cust_id)
    itm = BaseDBTransformer.readf(C.items, **{C.ordid: order_id})
    itm_d = BaseDBTransformer.readf(C.items, **{C.ordid: order_id_disc})
    crt = BaseDBTransformer.readf(C.cart, **{C.custid: cust_id})
    ord = BaseDBTransformer.readf(C.orders, **{C.ordid: order_id})
    ord_d = BaseDBTransformer.readf(C.orders, **{C.ordid: order_id_disc})


In [ ]:
disc_id = DiscountTransformer.generate_discount_id(order_id, cust_id)
dis = BaseDBTransformer.readf(C.discounts, **{C.did: disc_id})
print( len(dis), len(ord), len(crt), len(itm) )
assert all( [(len(dis) == 0), (len(itm) == 0), ( len(ord) == 0), ( len(crt) == 0)]) 
print( " Success Cleanup successfull after purchase ")

In [ ]:
DiscountService.list_eligible_codes(3)

# **Admin APIs**

### **Get all discount codes**

In [ ]:
all_disc_codes = dt.list_all_codes()
all_disc_codes


### **Get codes eligible - But not enabled**

In [ ]:
eligible_disc_codes = dt.list_eligible_codes(customer_id)
eligible_disc_codes

### **Get all active codes - But not yet used by the customer**

In [ ]:
active_disc_codes = dt.list_active_codes()
active_disc_codes

#### **Get used discount codes by the customer**

In [ ]:
dt.list_used_codes(customer_id)

### **Enable a discount code for a customer**

#### **First change the status**

In [ ]:
discount_id = "3__10071" 
dt.enable_discount_codes(discount_id, 20, "FESTIVAL OFFER")

#### **Second verify the record that status has really changed**

In [ ]:
active_disc_codes = dt.list_active_codes()
active_disc_codes

#### **Check Orders and items by customer**

In [ ]:
# print(order_id)
# print( PurchaseTransformer.list_orders_by_customer(customer_id) )
# df = BaseDBTransformer.read(C.items, "10077")
# df


### **Test the FASTAPIs**

In [ ]:
df = ProductTransformer.get_products_class()
mapped = df.to_dict(orient="records")
mapped

### **Invoices**

##### **Check all the tables and entries**

In [ ]:
prds_all = BaseDBTransformer.read(C.prd)
orders = BaseDBTransformer.read(C.orders)
items =  BaseDBTransformer.read(C.items)
prdc = BaseDBTransformer.read(C.prdc)
custs = BaseDBTransformer.read(C.custs)
print( custs['CUSTOMER_ID'].sort_values().unique())
print( items[C.pid].sort_values().unique())
print( prdc[C.ptyp].sort_values().unique() )
prds_all[C.pid].sort_values().unique()


##### **Generate the invoices**

In [ ]:
PurchaseService.generate_invoice(36)

## **Cleanup the database**

In [ ]:
# from pathlib import Path
# BaseDBTransformer.purge()

# ROOT = BaseDBTransformer.get_project_root()  # go up from tests/ to project root
# sql_path = ROOT / "data" / "ordersdb.sql"
# print( sql_path, "\n", ROOT)

In [ ]:
if(1):   
    # Get Products to buy -- Step 1 
    products_to_buy = ProductService.getRandomProducts()

    avail_dict_ori = products_to_buy.set_index(C.pid)[C.pavl].to_dict()
    print( avail_dict_ori )

    # First ask to buy more than avilable - It must fail 
    selected_items = products_to_buy.loc[:, [C.pid]]
    selected_items[C.qnt] = np.random.randint( products_to_buy.loc[:, [C.pavl]].max(), products_to_buy.loc[:, [C.pavl]].max() + 1000, size=len(selected_items))
    products = selected_items.to_dict(orient='records')

    # Add Them to Cart - Step 2 - Must fail as quantity is higher than available. 
    retval = CartService.addToCart(cust_id, products, True)
    if (retval < 0):
        print("Success Check avilability check verified. Higher quanitties check passed. ")
        assert(True)
    
    # Add the proper amounts to the cart. - Step 3
    selected_items = products_to_buy.loc[:, [C.pid]]
    selected_items[C.qnt] = np.random.randint(1, products_to_buy.loc[:, [C.pavl]].min(), size=len(selected_items))
    products = selected_items.to_dict(orient='records')
    retval = CartService.addToCart(cust_id, products, True)
    if (retval < 0):
        print("Something wrong with quantities. Even Minimum is failing. ", products)
        assert(False)
    else:
        read_back = CartService.get_customer_cart(cust_id).loc[:, [C.pid, C.qnt]]
        read_back_products = read_back.to_dict(orient='records')
        print(products)
        print(read_back_products)
        assert products == read_back_products, "Product Dictionaries dont match in the cart. Aborting purchase."
        print(" Success Verfied AddToCart ")

    # Initiate a Purchase - Step 4
    order_id = PurchaseService.purchase_cart_items(cust_id)
    print(order_id, "Generated from the purchase ")

    # Verifications - Step 5
    # First verify the items and order 
    read_back_items = BaseDBTransformer.readf(C.items, **{C.ordid: order_id}).loc[:, [C.pid, C.qnt]]
    print(read_back)
    assert products == read_back_items.to_dict(orient='records'), "Product Dictionaries dont match from order items. Cart items to Order items conversion failed - Purchase failed. ."

    # Next verify the quanitites of availability updatation.  - Step 6
    avail_dict_new = BaseDBTransformer.readdf(C.prd, C.pid, list(selected_items[C.pid])).set_index(C.pid)[C.pavl].to_dict()

    cart_quanties = selected_items.set_index(C.pid)[C.qnt].to_dict()
    
    print ( avail_dict_new, "\n", avail_dict_ori, "\n", cart_quanties)
    assert all(avail_dict_ori[k] - cart_quanties[k] == avail_dict_new[k] for k in avail_dict_ori)
    print(" Success Purchase cart is now verfied. For orders, order items and product quantities. ")

     # Verify if cart is now empty and Addition of discount coupons is successfull. 
    read_back = CartService.get_customer_cart(cust_id)
    if (len(read_back) > 0):
        print( "Cart NOT empty after purchase. Test failed. ")
        assert(False)
    
    disc_id = DiscountTransformer.generate_discount_id(order_id, cust_id)
    disc = BaseDBTransformer.readf(C.discounts, **{C.did: disc_id})
    if (len(disc) <= 0):
        print( "Discount code not found. Test failed. ")
        assert(False)
    
    scalars = disc.iloc[0].to_dict()
    status = scalars[C.dst]
    percent = scalars[C.dpct]
    oid = scalars[C.ordid]
    dcode = scalars[C.dcode]
    dcus = scalars[C.custid]

    print( scalars )
    print( status, percent, oid, dcode, dcus)

    assert all( [(status == 0), (percent == 0), ( dcus == cust_id), ( oid == order_id)]) 
    print( "Success - Purchase from cart without discount is completely verfied.")

    # Now make a purchase with discount using this dicount id created. 
    products_to_buy = ProductService.getRandomProducts()
    selected_items = products_to_buy.loc[:, [C.pid]]
    selected_items[C.qnt] = np.random.randint(1, products_to_buy.loc[:, [C.pavl]].min(), size=len(selected_items))
    products = selected_items.to_dict(orient='records')
    retval = CartService.addToCart(cust_id, products, True)
    if (retval < 0):
        print("Something wrong with quantities. Even Minimum is failing. ", products)
        assert(False)
    # Enable the discount coupon 
    DiscountService.enable_discount_codes(disc_id, 10, "FESTIVAL OFFER", True)
    order_id_disc = PurchaseService.purchase_discounted_cart_items(cust_id, disc_id)
    print(order_id_disc, "Generated from the purchase with discount ")

    # Verify if discount is set to used. 
    disc = BaseDBTransformer.readf(C.discounts, **{C.did: disc_id})
    if (len(disc) <= 0):
        print( "Discount code not found. Test failed. ")
        assert(False)
    scalars = disc.iloc[0].to_dict()
    status = scalars[C.dst]
    print( status, scalars)
    assert (status == 1)
    print( "Verified Discount coupon after use in purchase is now flagfed as used. (1) ")



In [ ]:
print(order_id_disc, "Generated from the purchase with discount ")
    status = scalars[C.dst]
    print( status, scalars)
    assert (status == 1)
    print( "Verified Discount coupon after use in purchase is now flagfed as used. (1) ")

In [ ]:
    # lastly cleanup this purchase and verify if delete purcahse is working. 
    PurchaseTransformer.deletePurchase(order_id_disc, cust_id, None)
    PurchaseService.deletePurchase(order_id, cust_id)
    itm = BaseDBTransformer.readf(C.items, **{C.ordid: order_id})
    itm_d = BaseDBTransformer.readf(C.items, **{C.ordid: order_id_disc})
    crt = BaseDBTransformer.readf(C.cart, **{C.custid: cust_id})
    ord = BaseDBTransformer.readf(C.orders, **{C.ordid: order_id})
    ord_d = BaseDBTransformer.readf(C.orders, **{C.ordid: order_id_disc})
    dis = BaseDBTransformer.readf(C.discounts, **{C.did: disc_id})
    print( len(dis), len(ord), len(crt), len(itm) )
    assert all( [(len(dis) == 0), (len(itm) == 0), ( len(ord) == 0), ( len(crt) == 0)]) 
    print( " Success Cleanup successfull after purchase ")

In [ ]:
dis = BaseDBTransformer.readf(C.discounts, **{C.did: disc_id})
order_id
dis

In [ ]:
cords = custs[C.custid].unique().tolist()
cords
do_agg[C.ordid]
